In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('../src')
import utils_explore as ue

data = ue.all_bets()

In [ ]:
df = ue.calc_win_rate(data, period_split='M', min_games_thres = 5)

In [ ]:
#plot success rate of some best authors
selected_authors = ['chaplygin'
                    , 'falcao1984'
                    , 'zhukov'
                    , 'stavkaprognozsport'
                    , 'teplofevralya']
df_plot = df[df.author.isin(selected_authors)] \
    [['author', 'placed-date', 'success_rate']]
df_plot = df_plot.pivot(index='placed-date', columns='author', values='success_rate')
df_plot.plot()

In [ ]:
#interpolate NANs
df_plot.interpolate().plot()